In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.baseline_functions as base

%store -r summary_violent_two_year_kentucky

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test


In [3]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
data = data.drop(['fta_risk_score_raw','nca_risk_score_raw','pvf_risk_score_raw', 
                  'fta_calc', 'nca_calc', 'pvf_calc'], axis=1)
x = data.loc[:,:'current_violence']
y = data['recid_violence_two_year'].values

In [5]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(X=x,
                                 Y=y,
                                 C=c,
                                 seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(X=x,
                           Y=y,
                           C=c,
                           seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(X=x,
                             Y=y,
                             C=c,
                             seed=816)

#### CART
depth = [8,9,10]
cart_summary = base.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### Random Forest
n_estimators =  [40,60,80]
depth = [8,9]
rf_summary = base.RF(X=x,
                     Y=y,
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.01]
depth = [8]
n_estimators = [150]
xgb_summary = base.XGB(X=x,
                       Y=y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators, 
                       seed=816)

#### save results
summary_violent_two_year_kentucky = {"Logistic": logistic_summary,
                         "Lasso": lasso_summary,
                         "LinearSVM": svm_summary,
                         "RF": rf_summary,
                         "XGBoost": xgb_summary}
%store summary_violent_two_year_kentucky

model_id, score_thresholds 0 {'rank_abs': [6262]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6400]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6381]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6318]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6234]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6271]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6411]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6399]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6316]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [6245]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [518]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [558]}
get_disparity_predefined_group()
model_

In [6]:
results = []
auc = [] ## for table writing -- the performance range 

for model_name, model_summary in summary_violent_two_year_kentucky.items():
    results.append([model_name, np.mean(model_summary['holdout_test_auc']), np.mean(model_summary['auc_diffs'])])
    auc.append(np.mean(model_summary['holdout_test_auc']))
    
results

In [8]:
params = [[model_name, model_summary['best_param']] for model_name, model_summary in summary_violent_two_year_kentucky.items()]
params

In [13]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\Baselines\\Two Year\\"
results = [["Violent", np.str((round(np.mean(logistic_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(logistic_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(lasso_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(lasso_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(svm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(svm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(rf_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(rf_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(xgb_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(xgb_summary['holdout_test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3)]]
with open(path + 'Two Year Baseline Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)